- Sometimes just the topic keywords may not be enough to make sense of what a topic is about. So, to help with understanding the topic,we can find the documents a given topic has contributed to the most and infer the topic by reading that document.
- The notebook also extracts the named entiies of person, organization and people in the relevant emails. It also exports the list of topics into csv files. For each topic, there is one csv file that contains dominant documents, the named entity of peron, the named entity of organization and the name of money hidden in the email content.

**Setup the google colab environment**

In [ ]:
from google.colab import drive
# This will prompt for authorization.
# authorization code: 4/OwErfUj6QceGXhIGx_RWv0MKclb9rilw8UsJnZqFbSez-QS8zQ399JU
drive.mount('/content/drive')

!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

!python -m spacy download en_core_web_md

**Import libararies**

In [ ]:
from joblib import load, dump
import pandas as pd
import os
from spacy import displacy
from collections import Counter
#from en_core_web_sm import load
from spacy.tokens import Span
from spacy.pipeline import EntityRuler
import en_core_web_md



In [ ]:
nlp = en_core_web_md.load()

**Google drive access path**

In [ ]:
metadata_path = '/content/drive/My Drive/Colab Notebooks/output/'

**Load the model, content and corpus**

In [ ]:
lda_model = load(metadata_path + 'lda_model.joblib')
corpus = load(metadata_path + 'corpus.joblib')

topic_csv_fname = 'mail_topics.csv'

In [ ]:
def get_money_entity(doc):
    money_ents = Counter([ent.text for ent in doc.ents if ent.label_ == 'MONEY'])
    #return list(money_ents)
    return ' '.join(money_ents)

def get_organization_entity(doc):
    org_ents = Counter([ent.text for ent in doc.ents if ent.label_ == 'ORG'])
    # return list(join(org_ents))
    return ' '.join(org_ents)

def get_person_entity(doc):
    person_ents = Counter([ent.text for ent in doc.ents if ent.label_ == 'PERSON'])
    return ' '.join(person_ents

**1) Export the topic list into csv files**

In [ ]:
topics = lda_model.print_topics(50)

df_topics = pd.DataFrame(data = {'key_words' : topics}
                         , index = range(len(topics)) )

df_topics.to_csv(metadata_path + topic_csv_fname)

**2) Find dominant documents to each category**

In [ ]:
# Assigns the topics to the documents in corpus
lda_corpus = lda_model[corpus]
# find the dominant topics
lda_corpus = [max(prob, key = lambda y : y[1]) for prob in lda_corpus ]
list_content_by_topic = [[] for i in range(50)]
# select the most relevant documents to the topic
for i, x in enumerate(lda_corpus):
    #print(x[0])
    list_content_by_topic[x[0]].append(list_content[i])

**3) Extrac the named enitiy of person, organization and money in the relevant emails. Finally, save them all as csv files**

In [ ]:
for idx, content in enumerate(list_content_by_topic):

    for text in content:
        doc = nlp(text)
        list_money_ents = get_money_entity(doc)
        list_person_ents = get_person_entity(doc)
        list_org_ents = get_organization_entity(doc)

    df_content = pd.DataFrame(data = {'mail_content' : content 
                                      , 'person_entity': list_person_ents
                                      , 'money_entity' : list_money_ents
                                      , 'organisation_entity' : list_org_ents}
                                       , index = range(len(content)))
    
    content_csv_fname = "mail_content_by_topics_" + str(idx) + ".csv"

    df_content.to_csv(metadata_path + content_csv_fname)